In [1]:
# File Listener and Database Upload
# ---------------------------------------------------
# This script watches a folder for new CSV files and
# uploads their content into a SQLite database.

In [2]:
import os
import time
import pandas as pd
import sqlite3

In [3]:
# Constants
WATCH_DIR = "watched_folder"
DB_FILE = "sales_database.db"
TABLE_NAME = "sales"
PROCESSED_DIR = "processed_files"

In [4]:
# Ensure folders exist
os.makedirs(WATCH_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)

In [5]:
# Define expected columns
EXPECTED_COLUMNS = ["ID", "Date", "Product", "Category", "Quantity", "Price", "Total"]


In [6]:
# Connect to SQLite database
def connect_db():
    conn = sqlite3.connect(DB_FILE)
    return conn

In [7]:
# Create table if not exists
def create_table():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
            ID INTEGER,
            Date TEXT,
            Product TEXT,
            Category TEXT,
            Quantity INTEGER,
            Price REAL,
            Total REAL
        )
    ''')
    conn.commit()
    conn.close()

In [8]:
# Insert DataFrame into database
def upload_to_database(filepath):
    try:
        df = pd.read_csv(filepath)
        if not all(col in df.columns for col in EXPECTED_COLUMNS):
            print(f"Skipping {filepath}: Columns do not match.")
            return

        conn = connect_db()
        df.to_sql(TABLE_NAME, conn, if_exists='append', index=False)
        conn.close()

        print(f"Uploaded: {filepath}")
        os.rename(filepath, os.path.join(PROCESSED_DIR, os.path.basename(filepath)))
    except Exception as e:
        print(f"Error uploading {filepath}: {str(e)}")

In [9]:
# Monitor folder for new files
def listen_for_files():
    print(f"Watching folder: {WATCH_DIR}... Press Ctrl+C to stop.")
    processed = set(os.listdir(PROCESSED_DIR))

    while True:
        try:
            for filename in os.listdir(WATCH_DIR):
                if filename.endswith(".csv") and filename not in processed:
                    full_path = os.path.join(WATCH_DIR, filename)
                    upload_to_database(full_path)
                    processed.add(filename)
            time.sleep(5)
        except KeyboardInterrupt:
            print("\nStopped file listener.")
            break

In [ ]:
# Entry point
if __name__ == "__main__":
    create_table()
    listen_for_files()


Watching folder: watched_folder... Press Ctrl+C to stop.
